In [1]:
from utils.video_dataset import VideoFrameDataset
from config.torch_config import multiple_transform, device
from config.dataset import get_dataset_path


In [2]:
NUM_SEGMENTS = 1
FRAMES_PER_SEGMENT = 5
BATCH_SIZE = 8

In [3]:
data_path, model_path = get_dataset_path()


In [4]:
dataset = VideoFrameDataset(
    root_path=data_path,
    transform=multiple_transform,
    num_segments=NUM_SEGMENTS,
    frames_per_segment=FRAMES_PER_SEGMENT,
)

classes = dataset.classes


In [5]:
from utils.loader import split_dataset


In [6]:
train_loader, test_loader, validation_loader = split_dataset(
    dataset, train_split=0.7, validation_split=0.1, batch_size=BATCH_SIZE
)


In [7]:
print(len(train_loader), len(validation_loader), len(test_loader))


25 4 8


In [8]:
from utils.balance import check_balance_status

In [9]:
# check_balance_status(test_loader, classes)
# check_balance_status(train_loader, classes)


In [10]:
from lib.simple_model import HPNet


In [11]:
model = HPNet(
    num_classes=len(classes),
    batch_size=BATCH_SIZE,
    num_frames=FRAMES_PER_SEGMENT,
    image_size=224,
)
model

HPNet(
  (conv3d_1): Sequential(
    (0): Conv3d(5, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (conv3d_2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (batch): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (flat): Flatten(start_dim=2, end_dim=-1)
  (lin_1): Sequential(
    (0): Linear(in_features=3136, out_features=1568, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (lin_2): Sequential(
    (0): Linear(in_features=1568, out_features=784, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (lin_3): Linear(in_features=784, out_features=10, bias=True)
  (soft): Softmax(dim=1)
  (drop): Dropout(p=0.15, inplace=False)
)

In [12]:
from lib.train import train_model

In [13]:
costs = train_model(
    model, train_loader, validation_loader, device, learning_rate=0.001, num_epochs=5
)


Training on device: cuda
Train cost at epoch 1 is 5.66726
Validation cost at epoch 1 is 5.66726
Train cost at epoch 2 is 5.48796
Validation cost at epoch 2 is 5.48796
Train cost at epoch 3 is 5.48796
Validation cost at epoch 3 is 5.48796
Train cost at epoch 4 is 5.48796
Validation cost at epoch 4 is 5.48796
Train cost at epoch 5 is 5.48796
Validation cost at epoch 5 is 5.48796


In [14]:
from lib.test import check_accuracy

In [15]:
check_accuracy(train_loader, model, classes, device)

KeyboardInterrupt: 